In [1]:
import numpy as np
import pandas as pd

from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model, save_model
import pickle

import warnings 
warnings.filterwarnings('ignore')

In [2]:
model = load_model("Stocks.h5")
data=pd.read_csv(r"C:\Users\Priyanshul\OneDrive\Desktop\Trading Bot\INFO.csv")
with open(r"C:\Users\Priyanshul\OneDrive\Desktop\Trading Bot\scaler1.pkl", "rb") as f:
    scaler1 = pickle.load(f)
with open(r"C:\Users\Priyanshul\OneDrive\Desktop\Trading Bot\scaler2.pkl", "rb") as f:
    scaler2 = pickle.load(f)

In [3]:
data.index=data['Date']
data.drop('Date', axis=1, inplace=True)
test_req=data.iloc[1803:1863,0:7]

In [4]:
def clean_text(text):
    text = re.sub("b[(')]",' ', text)
    text = re.sub('b[(")]',' ', text)
    text = re.sub("\'",' ', text)
    return text

In [5]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [6]:
def predict_stock_close_price(open_price, headline, merged_df_test):
    # Clean the headline
    clean_headline = clean_text(headline)

    # Get subjectivity and polarity
    subjectivity = getSubjectivity(clean_headline)
    polarity = getPolarity(clean_headline)

    # Get sentiment scores
    SIA = getSIA(clean_headline)
    compound = SIA['compound']
    negative = SIA['neg']
    neutral = SIA['neu']
    positive = SIA['pos']

    # Append the new data to the test dataframe
    new_data = pd.DataFrame([[open_price, subjectivity, polarity, compound, negative, neutral, positive]], columns=merged_df_test.columns)
    merged_df_test = merged_df_test.append(new_data, ignore_index=True)
    
    # Create a 60 days window for the input data
    test_x = []
    for i in range(60, merged_df_test.shape[0]):
        test_x.append(merged_df_test.iloc[i-60:i, 0:7].values)  
    test_x= np.array(test_x)

    inputs_test  = scaler1.transform(test_x[:,:,0])
    inputs_test= inputs_test.reshape(1,60,1)
    dummy_test=test_x[:,:,1:7]
    inputs_test = np.concatenate((inputs_test,dummy_test), axis=2)
    # Make prediction
    closing_price_test = model.predict(inputs_test)
    closing_price_test = scaler2.inverse_transform(closing_price_test)

    window_size = 60  # Define your window size
    ma_slow = np.mean(merged_df_test['Open'][-window_size:])  # Slow moving average
    ma_fast = np.mean(merged_df_test['Open'][-window_size//2:])  # Fast moving average

    # Determine buy and sell signals
    if ma_fast > ma_slow:
        action = 'Buy'
    else:
        action = 'Sell'

    return {'Predicted_close_price': closing_price_test, 'Action': action}

In [7]:
open_price=17113.55078
headline='b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\' b\'Rice Gives Green Light for Israel to Attack Iran: Says U.S. has no veto over Israeli military ops\' b\'Announcing:Class Action Lawsuit on Behalf of American Public Against the FBI\' b"So---Russia and Georgia are at war and the NYT\'s top story is opening ceremonies of the Olympics?  What a fucking disgrace and yet further proof of the decline of journalism." b"China tells Bush to stay out of other countries\' affairs" b\'Did World War III start today?\' b\'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?\' b\'Al-Qaeda Faces Islamist Backlash\' b\'Condoleezza Rice: "The US would not act to prevent an Israeli strike on Iran." Israeli Defense Minister Ehud Barak: "Israel is prepared for uncompromising victory in the case of military hostilities."\' b\'This is a busy day:  The European Union has approved new sanctions against Iran in protest at its nuclear programme.\' b"Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia\'s breakaway region of South Ossetia" b\'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report\' b\'Caucasus in crisis: Georgia invades South Ossetia\' b\'Indian shoe manufactory  - And again in a series of "you do not like your work?"\' b\'Visitors Suffering from Mental Illnesses Banned from Olympics\' b"No Help for Mexico\'s Kidnapping Surge"'

In [8]:
result=predict_stock_close_price(open_price, headline, test_req)
result

1/1 [==============================] - 1s 1s/step


{'Predicted_close_price': array([[17581.467]], dtype=float32), 'Action': 'Buy'}